## Import Libraries

In [1]:
import chess
from stockfish import Stockfish
import re
import requests
import pandas as pd
import random


## Instantialize the stockfish engine

In [2]:
stockfish = Stockfish()

## Copy data from the CSV file

In [3]:
csv_file = "/Users/nafis-mac/GitHub Repos/Chess-Research-Project/puzzles_code/filtered_fen.csv"

# Read the first 100 rows of the CSV file
df = pd.read_csv(csv_file, nrows=100)

# Convert the DataFrame rows to a list of lists
fen_notations = df["FEN"].tolist()

print(len(fen_notations))

100


## Create function for random move generation

In [4]:
def get_random_move(board:chess.Board, fen:str, seed: int = 101):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
    ]

    random.seed(seed)

    # print(random.choice(legal_moves))
    return random.choice(legal_moves)

    # print(legal_moves[0])

### Sanity check for random move generator

In [5]:
board = chess.Board(fen=fen_notations[0])

get_random_move(board,fen_notations[0])

'd3'

## List for saving results

In [6]:
results = []

## Getting the best move and random move for multiple iterations for each puzzle

In [ ]:
puzzle_id = 1


for fen in fen_notations:
    for i in range (50):
        print(f"Processing puzzle_id: {puzzle_id} Iteration: {i+1}\nPosition: {fen}")

        board_stockfish = chess.Board(fen=fen)
        board_random = chess.Board(fen=fen)

        assert board_random == board_stockfish, "Two boards are not equal"

        stockfish.set_fen_position(fen_position=fen)

        best_move = stockfish.get_best_move()

        board_stockfish.push(chess.Move.from_uci(best_move))
        stockfish.set_fen_position(fen_position=board_stockfish.fen())
        best_eval = stockfish.get_evaluation()  # evaluation in centipawns (or mate score)
        
        # print(f"Stockfish best move: {best_move} with evaluation: {best_eval}")
        if best_eval["type"] == "mate":
            break

        random_move = get_random_move(board_random, fen, seed=i)

        try:
            board_random.push_san(random_move)
        except Exception as e:
            print(f"Exception captured: {type(e)}")

        stockfish.set_fen_position(fen_position=board_random.fen())
        provided_eval = stockfish.get_evaluation()

        if provided_eval["type"] == "mate":
            print("ITERATION SKIPPED")
            continue

        score_difference = (best_eval["value"]) - (provided_eval["value"])

        print(f"Stockfish best move: {best_move} with evaluation: {best_eval["value"]}")
        print(f"Random move: {random_move} with evaluation: {provided_eval["value"]}\n")

        results.append({
            "Iteration": i+1,
            "Puzzle ID": puzzle_id,
            "Method": "Random",
            "Best Move Score": best_eval["value"],
            "Move made by Random Score": provided_eval["value"],
            # "Score Difference": score_difference
        })
    
    puzzle_id += 1

Processing puzzle_id: 1 Iteration: 1
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Random move: b4 with evaluation: 21

Processing puzzle_id: 1 Iteration: 2
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Random move: Bc4 with evaluation: -473

Processing puzzle_id: 1 Iteration: 3
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Random move: b4 with evaluation: 21

Processing puzzle_id: 1 Iteration: 4
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Random move: Ke2 with evaluation: -52

Processing puzzle_id: 1 Iteration: 5
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Random move: Ke2 with evaluation: -52

Processing puzzle_id: 1 Iteration

KeyboardInterrupt: 

## Saving data to the CSV file

In [11]:
df = pd.DataFrame(results)
df.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")

Results saved to evaluation_results.csv
